# Content Based

## Created by : Andreas MM
<br><br>
Now, we tried to give recomendation movies to user **Mulya** based on Content Based. Basicaly, we calculate cosine similarities between movies based on its genre and give recomendation to **Mulya** based on movie that already watched by him. If there are a lot of similarities between those movies, we recomend it to him. Basicaly, our line of thoughts is :
1. Data preparation and Data Merging
2. Use CountVectorizer (noted there are also TfidVectorizer but TfidVectorizer also calculated the weights from frequency and how frequent those words are while CountVectorizer just count the word in strings) to labeled and count these features and labeled each words. (we compared these vectorizer below)
3. calculate cosine similarities
4. Input movie that already been watched
5. sort and find best 3 movie that are similar to already watched movies. Those 3 was recomendation movies to user 
<br>
<br>

### Data Preparation

In [2]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [3]:
data = pd.read_csv('Movie Genre.csv',delimiter=';')
data.head()

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,...,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response
0,1,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True
1,2,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True
2,3,The Godfather: Part II,1974,R,20 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola (screenplay), Mario Puzo ...","Al Pacino, Robert Duvall, Diane Keaton, Robert...",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24 May 2005,NaN,Paramount Pictures,http://www.thegodfather.com/,True
3,4,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True
4,5,12 Angry Men,1957,APPROVED,01 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,"Reginald Rose (story), Reginald Rose (screenplay)","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06 Mar 2001,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True


In [4]:
data.columns

Index(['Unnamed: 0', 'Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre',
       'Director', 'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards',
       'Poster', 'Ratings.Source', 'Ratings.Value', 'Metascore', 'imdbRating',
       'imdbVotes', 'imdbID', 'Type', 'tomatoMeter', 'tomatoImage',
       'tomatoRating', 'tomatoReviews', 'tomatoFresh', 'tomatoRotten',
       'tomatoConsensus', 'tomatoUserMeter', 'tomatoUserRating',
       'tomatoUserReviews', 'tomatoURL', 'DVD', 'BoxOffice', 'Production',
       'Website', 'Response'],
      dtype='object')

<br><br><br>
We have 250 films and 38 rows, with each columns can be seen above. We only need few of them that we think is cirelated to the feature of movies. That is Title, Rated, Runtime, Genre, Language, Country,Plot,Director,Actors. 

In [5]:
data2=data[['Title','Rated','Runtime','Genre','Language','Country','Plot','Director','Actors']]
data2=data2.set_index('Title')
del data2.index.name

<br><br>
Then, we join our data in to one column. Noted that in data above, there are commas between data. We need to remove those using regex in pandas.

In [6]:
data2['Join']=data2.apply(lambda x: ' '.join(x), axis = 1)
data2=data2.replace(regex=True,to_replace=r',',value=r'')['Join']
data2=pd.DataFrame(data2)
data2.head()

,Join
The Shawshank Redemption,R 142 min Crime Drama English USA Two imprison...
The Godfather,R 175 min Crime Drama English Italian Latin US...
The Godfather: Part II,R 202 min Crime Drama English Italian Spanish ...
The Dark Knight,PG-13 152 min Action Crime Drama English Manda...
12 Angry Men,APPROVED 96 min Crime Drama English USA A jury...


### Count Vectorizer
After we merge those columns. We use CountVectorizer to count and labeled our data. We also claculated tfidfVectorizer to compared them. 

In [7]:
count = CountVectorizer()
count_matrix = count.fit_transform(data2['Join'])

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(data2['Join'])

# generating the cosine similarity matrix
ind=list(data2.index)
cor=pd.DataFrame(cosine_similarity(count_matrix, count_matrix),index=ind,columns=ind)
cor.index=ind
cor.columns=ind

cor_t=pd.DataFrame(cosine_similarity(tfidf_matrix,tfidf_matrix),index=ind,columns=ind)


In [8]:
cor.head()

,The Shawshank Redemption,The Godfather,The Godfather: Part II,The Dark Knight,12 Angry Men,Schindler's List,The Lord of the Rings: The Return of the King,Pulp Fiction,Fight Club,The Lord of the Rings: The Fellowship of the Ring,...,Deadpool,The Wild Bunch,Aladdin,Big Fish,Patton,The Lost Weekend,Short Term 12,His Girl Friday,The Straight Story,Slumdog Millionaire
The Shawshank Redemption,1.000000,0.254164,0.215166,0.225891,0.200082,0.097243,0.213395,0.404762,0.093352,0.130189,...,0.177915,0.160128,0.145865,0.142915,0.188562,0.203091,0.231455,0.111111,0.106752,0.184900
The Godfather,0.254164,1.000000,0.472500,0.349780,0.339993,0.155709,0.234305,0.283212,0.192187,0.158829,...,0.139535,0.195355,0.177954,0.235380,0.258799,0.185826,0.211779,0.203331,0.219774,0.274921
The Godfather: Part II,0.215166,0.472500,1.000000,0.390327,0.243544,0.169480,0.281001,0.313527,0.126543,0.268917,...,0.157500,0.206725,0.131818,0.243544,0.273861,0.202260,0.199205,0.193649,0.124035,0.286446
The Dark Knight,0.225891,0.349780,0.390327,1.000000,0.303959,0.167282,0.400463,0.342559,0.116791,0.420767,...,0.174890,0.233723,0.243320,0.268199,0.383349,0.217786,0.273483,0.225891,0.133556,0.404820
12 Angry Men,0.200082,0.339993,0.243544,0.303959,1.000000,0.175109,0.241540,0.244998,0.168102,0.200945,...,0.156920,0.137309,0.250156,0.235294,0.218282,0.179124,0.238165,0.257248,0.219694,0.261608


In [9]:
cor_t.head()

,The Shawshank Redemption,The Godfather,The Godfather: Part II,The Dark Knight,12 Angry Men,Schindler's List,The Lord of the Rings: The Return of the King,Pulp Fiction,Fight Club,The Lord of the Rings: The Fellowship of the Ring,...,Deadpool,The Wild Bunch,Aladdin,Big Fish,Patton,The Lost Weekend,Short Term 12,His Girl Friday,The Straight Story,Slumdog Millionaire
The Shawshank Redemption,1.000000,0.040322,0.035432,0.036429,0.024948,0.006343,0.046275,0.156188,0.005630,0.013050,...,0.037287,0.029803,0.037416,0.035224,0.022942,0.040147,0.022562,0.007270,0.007035,0.019399
The Godfather,0.040322,1.000000,0.278020,0.067743,0.056117,0.017779,0.029701,0.048791,0.029662,0.017005,...,0.014380,0.057939,0.019789,0.052505,0.051794,0.018962,0.022205,0.023915,0.037066,0.036405
The Godfather: Part II,0.035432,0.278020,1.000000,0.087996,0.041744,0.021321,0.049420,0.063754,0.023531,0.052974,...,0.032911,0.055178,0.014595,0.065816,0.099526,0.039357,0.021595,0.024438,0.014974,0.053449
The Dark Knight,0.036429,0.067743,0.087996,1.000000,0.048582,0.021313,0.091620,0.059567,0.029196,0.115563,...,0.038936,0.058425,0.055368,0.054364,0.070271,0.026912,0.034931,0.035076,0.022707,0.087778
12 Angry Men,0.024948,0.056117,0.041744,0.048582,1.000000,0.025213,0.026374,0.042564,0.015877,0.021041,...,0.014670,0.011587,0.034774,0.025094,0.025181,0.015533,0.058797,0.038179,0.032291,0.029663


<br><br><br>
We see that CountVectorizer is more sense than TfidVectorizer in our case as for example, The GodFather and The GodFather part II, the result of cosine in countvectorizer is higher than tfidvectorizer. You can also see it in others prequels moviesl. Because of it, we only use CountVectorizer

<br><br>
Now, we input what movies that the user already watched 

In [43]:
movie_watched = ['Interstellar',
                 'The Martian',
                'Guardians of the Galaxy']
print('Movie already Watched : ',movie_watched)

Movie already Watched :  ['Interstellar', 'The Martian', 'Guardians of the Galaxy']


### Content Based 
Using cosine similarities, we average the cosine corelation of other movies and movie that already watched and take best three to recomend to user

In [35]:
rec=cor[movie_watched]
rec2=pd.DataFrame(rec.mean(axis=1),columns=['Score']).sort_values('Score',ascending=False).iloc[len(movie_watched):,:]

print('Film Recomendation : \n',list(rec2.head().index)[:3])

Film Recomendation : 
 ['Guardians of the Galaxy Vol. 2', 'The Avengers', 'The Prestige']


In [36]:
rec2.head()

,Score
Guardians of the Galaxy Vol. 2,0.361956
The Avengers,0.322013
The Prestige,0.317291
Inception,0.300666
Blade Runner,0.294664


<br><br>
Using Content Based, based on movies already watched that is Interstelar and The Martian and Guardian of The Galaxy, we find that COntent Based algorithm will recomend Guardian of The Galaxy vol.2, Avengers, and The Prestige. We want to find why 

In [62]:

data3=data[['Title','Rated','Runtime','Genre','Language','Country','Plot','Director','Actors']]
movi=pd.DataFrame(columns=data3.columns)
for i in range(250):
    if (data3.iloc[i,0] in movie_watched) :
        movi=movi.append([data3.iloc[i,:]])
for i in range(250):
    if (data3.iloc[i,0] in list(rec2.head().index)[:3]):
        movi=movi.append([data3.iloc[i,:]])
movi

,Title,Rated,Runtime,Genre,Language,Country,Plot,Director,Actors
25,Interstellar,PG-13,169 min,"Adventure, Drama, Sci-Fi",English,"USA, UK, Canada, Iceland",A team of explorers travel through a wormhole ...,Christopher Nolan,"Ellen Burstyn, Matthew McConaughey, Mackenzie ..."
199,Guardians of the Galaxy,PG-13,121 min,"Action, Adventure, Sci-Fi",English,"USA, UK",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Zoe Saldana, Dave Bautista, Vin D..."
222,The Martian,PG-13,144 min,"Adventure, Drama, Sci-Fi","English, Mandarin","USA, UK",An astronaut becomes stranded on Mars after hi...,Ridley Scott,"Matt Damon, Jessica Chastain, Kristen Wiig, Je..."
40,The Prestige,PG-13,130 min,"Drama, Mystery, Sci-Fi",English,"USA, UK",Two stage magicians engage in competitive one-...,Christopher Nolan,"Hugh Jackman, Christian Bale, Michael Caine, P..."
203,Guardians of the Galaxy Vol. 2,PG-13,136 min,"Action, Adventure, Sci-Fi",English,USA,The Guardians must fight to keep their newfoun...,James Gunn,"Chris Pratt, Zoe Saldana, Dave Bautista, Vin D..."
219,The Avengers,PG-13,143 min,"Action, Sci-Fi","English, Russian, Hindi",USA,Earth's mightiest heroes must come together an...,Joss Whedon,"Robert Downey Jr., Chris Evans, Mark Ruffalo, ..."


## Conclusion
<br><br>
We found that these film, is All PG-13, English, USA/UK and has if not the same Director, then the same Actors. So it make sense the recomender system recomended those movies. Several improvement need to be made to improve these result (for example, by using other columns or removing Runtime etc)



We find that using Content based, if you already watched Interstelar and The Martian and Guardian of The Galaxy, COntent Based algorithm will recomend Guardian of The Galaxy vol.2, Avengers, and The Prestige. 